In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmpn6xy971q/swift-install
/home/qtran/swift/usr/bin/swift-build: /home/qtran/anaconda3/lib/libuuid.so.1: no version information available (required by /home/qtran/swift/usr/lib/swift/linux/libFoundation.so)
/home/qtran/swift/usr/bin/swift-build: /home/qtran/anaconda3/lib/libcurl.so.4: no version information available (required by /home/qtran/swift/usr/lib/swift/linux/libFoundation.so)
/home/qtran/swift/usr/bin/swiftc: /home/qtran/anaconda3/lib/libuuid.so.1: no version information available (required by /home/qtran/swift/usr/bin/swiftc)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
/home/qtran/swift/usr/bin/swiftc: /home/qtran/anaconda3/lib/libuuid.so.1: no version inform

Currently there's a bug in swift-jupyter which requires we define any custom operators here:

In [2]:
// export
precedencegroup ExponentiationPrecedence {
    associativity: right
    higherThan: MultiplicationPrecedence
}
infix operator ** : ExponentiationPrecedence

precedencegroup CompositionPrecedence { associativity: left }
infix operator >| : CompositionPrecedence

## Getting the MNIST dataset

- [Just](https://github.com/JustHTTP/Just) does the equivalent of requests in python
- [Path](https://github.com/mxcl/Path.swift) is even better than its python counterpart
- Foundation is the base library from Apple

In [4]:
//export
import Foundation
import Just
import Path

We will need to gunzip, untar or unzip files we download, so instead of grabbing one library for each, we implement a function that can execute any shell command.

In [4]:
public extension String {
    @discardableResult
    func shell(_ args: String...) -> String
    {
        let (task,pipe) = (Process(),Pipe())
        task.executableURL = URL(fileURLWithPath: self)
        (task.arguments,task.standardOutput) = (args,pipe)
        do    { try task.run() }
        catch { print("Unexpected error: \(error).") }

        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        return String(data: data, encoding: String.Encoding.utf8) ?? ""
    }
}

In [5]:
print("/bin/ls".shell("-lh"))

total 216K
-rw-rw-r-- 1 qtran qtran  55K Jun 17 22:20 load_data_and_tensor.ipynb
-rw-rw-r-- 1 qtran qtran  76K Jun 17 22:44 swift_basic.ipynb
-rw-rw-r-- 1 qtran qtran  80K Jun 17 22:21 Swift for TensorFlow_ walkthrough
drwxr-xr-x 4 qtran qtran 4.0K Jun 17 22:45 swift-install



To download a file, we use the `Just` library.

In [6]:
//export
public func downloadFile(_ url: String, dest: String? = nil, force: Bool = false) {
    let dest_name = dest ?? (Path.cwd/url.split(separator: "/").last!).string
    let url_dest = URL(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if !force && Path(dest_name)!.exists { return }

    print("Downloading \(url)...")

    if let cts = Just.get(url).content {
        do    {try cts.write(to: URL(fileURLWithPath:dest_name))}
        catch {print("Can't write to \(url_dest).\n\(error)")}
    } else {
        print("Can't reach \(url)")
    }
}

In [0]:
// downloadFile("https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz")

In [7]:
//export
import TensorFlow

The following is generic over the element type on the return value.  We could define two functions like this:

```swift
func loadMNIST(training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<Float> {
func loadMNIST(training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<Int32> {
```

but that would be boring.  So we make loadMNIST take a "type parameter" `T` which indicates what sort of element type to load a tensor into.

In [0]:
// func loadMNIST<T>(training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<T> {
//     let split = training ? "train" : "t10k" //name of train folder or test (t10k) folder
//     let kind = labels ? "labels" : "images" //image file or label (class) file
//     let batch = training ? 60000 : 10000
//     let shape: TensorShape = labels ? [batch] : (flat ? [batch, 784] : [batch, 28, 28]) // shape [60000] for label, [60k,784] or [60k,28,28] for img
//     let dropK = labels ? 8 : 16 // ?
//     let baseUrl = "https://storage.googleapis.com/cvdf-datasets/mnist/"
//     let fname = split + "-" + kind + "-idx\(labels ? 1 : 3)-ubyte"
//     let file = path/fname
//     if !file.exists { //download correct file based on fname component defined above
//         downloadFile("\(baseUrl)\(fname).gz", dest:(path/"\(fname).gz").string)
//         "/bin/gunzip".shell("-fq", (path/"\(fname).gz").string)
//     }
//     let data = try! Data(contentsOf: URL(fileURLWithPath: file.string)).dropFirst(dropK)
//     if labels { return Tensor(data.map(T.init)) }
//     else      { return Tensor(data.map(T.init)).reshaped(to: shape)}
// }

But this doesn't work because S4TF can't just put any type of data inside a `Tensor`. We have to tell it that this type:
- is a type that TF can understand and deal with
- is a type that can be applied to the data we read in the byte format

We do this by defining a protocol called `ConvertibleFromByte` that inherits from `TensorFlowScalar`. That takes care of the first requirement. The second requirement is dealt with by asking for an `init` method that takes `UInt8`:

In [8]:
//export
protocol ConvertibleFromByte: TensorFlowScalar {
    init(_ d:UInt8)
}

Then we need to say that `Float` and `Int32` conform to that protocol. They already have the right initializer so we don't have to code anything.

In [9]:
//export
extension Float : ConvertibleFromByte {}
extension Int32 : ConvertibleFromByte {}

Lastly, we write a convenience method for all types that conform to the `ConvertibleFromByte` protocol, that will convert some raw data to a `Tensor` of that type.

In [10]:
//export
extension Data {
    func asTensor<T:ConvertibleFromByte>() -> Tensor<T> {
        return Tensor(map(T.init))
    }
}

And now we can write a generic `loadMNIST` function that can returns tensors of `Float` or `Int32`.

In [11]:
//export
func loadMNIST<T: ConvertibleFromByte>
            (training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<T> {
    let split = training ? "train" : "t10k" //name of train folder or test (t10k) folder
    let kind = labels ? "labels" : "images" //image file or label (class) file
    let batch = training ? 60000 : 10000
    let shape: TensorShape = labels ? [batch] : (flat ? [batch, 784] : [batch, 28, 28])
    let dropK = labels ? 8 : 16
    let baseUrl = "https://storage.googleapis.com/cvdf-datasets/mnist/"
    let fname = split + "-" + kind + "-idx\(labels ? 1 : 3)-ubyte"
    let file = path/fname
    if !file.exists { //download correct file based on fname component defined above
        downloadFile("\(baseUrl)\(fname).gz", dest:(path/"\(fname).gz").string)
        "/bin/gunzip".shell("-fq", (path/"\(fname).gz").string)
    }
    let data = try! Data(contentsOf: URL(fileURLWithPath: file.string)).dropFirst(dropK)
    if labels { return data.asTensor() }
    else      { return data.asTensor().reshaped(to: shape)}
}

public func loadMNIST(path:Path, flat:Bool = false)
        -> (Tensor<Float>, Tensor<Int32>, Tensor<Float>, Tensor<Int32>) {
    try! path.mkdir(.p)
    return (
        loadMNIST(training: true,  labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: true,  labels: true,  path: path, flat: flat),
        loadMNIST(training: false, labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: false, labels: true,  path: path, flat: flat)
    )
}

We will store mnist in this folder so that we don't download it each time we run a notebook:

In [12]:
//export
public let mnistPath = Path.home/".fastai"/"data"/"mnist_tst"

In [13]:
print(mnistPath)

/home/qtran/.fastai/data/mnist_tst


The default returns mnist in the image format:

In [14]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath)
xTrain.shape

2019-06-17 22:46:43.087880: W tensorflow/core/framework/allocator.cc:122] Allocation of 188160000 exceeds 10% of system memory.
2019-06-17 22:46:43.261133: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-06-17 22:46:43.312517: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 1696090000 Hz
2019-06-17 22:46:43.321528: W tensorflow/core/framework/allocator.cc:122] Allocation of 188160000 exceeds 10% of system memory.


▿ [60000, 28, 28]
  ▿ dimensions : 3 elements
    - 0 : 60000
    - 1 : 28
    - 2 : 28


We can also ask for it in its flattened form:

In [15]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath, flat: true)
xTrain.shape

2019-06-17 22:46:57.395684: W tensorflow/core/framework/allocator.cc:122] Allocation of 188160000 exceeds 10% of system memory.
2019-06-17 22:46:57.619604: W tensorflow/core/framework/allocator.cc:122] Allocation of 188160000 exceeds 10% of system memory.


▿ [60000, 784]
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 784


In [16]:
print(type(of: xTrain))
print(type(of: yTrain))

Tensor<Float>
Tensor<Int32>


## Timing

Here is our time function:

In [17]:
//export 

//TODO


import Dispatch

// ⏰Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func time(repeating: Int = 1, _ f: () -> ()) {
    guard repeating > 0 else { return }
    
    // Warmup
    if repeating > 1 { f() }
    
    var times = [Double]()
    for _ in 1...repeating {
        let start = DispatchTime.now()
        f()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("average: \(times.reduce(0.0, +)/Double(times.count)) ms,   " +
          "min: \(times.reduce(times[0], min)) ms,   " +
          "max: \(times.reduce(times[0], max)) ms")
}

In [18]:
time(repeating: 10) {
    _ = loadMNIST(training: false, labels: false, path: mnistPath, flat: false) as Tensor<Float>
}

average: 416.30038449999995 ms,   min: 413.60394 ms,   max: 421.767481 ms


# Experiment with Tensor 

In [19]:
let zeros = Tensor<Float>(zeros: [1,4,5])
let ones  = Tensor<Float>(ones: [12,4,5])
let twos  = Tensor<Float>(repeating: 2.0, shape: [2,3,4,5])
let range = Tensor<Int32>(rangeFrom: 0, to: 32, stride: 1)

In [20]:
print(twos.shape)

[2, 3, 4, 5]


In [0]:
print(type(of: twos))
print(type(of: twos.shape))

Tensor<Float>
TensorShape


In [0]:
print(range)

[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [21]:
zeros[0]+1

[[1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0]]


In [22]:
let xTrain = Tensor<Float>(randomNormal: [5, 784])
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784) // Xavier
print(weights[0])

[-0.014152822, -0.033161078,  0.009075283,  0.003997042,  -0.06525668,  0.041842468,
  0.027588418,  0.015348827, -0.055493403, -0.036986433]


In [23]:
let temp = matmul(xTrain,weights) //built in matmul, not the one we created

In [24]:
let temp = xTrain • weights

In [25]:
time(repeating: 100) {
    _ = matmul(xTrain,weights)
}

average: 0.01866068 ms,   min: 0.017822 ms,   max: 0.050638 ms


In [26]:
let xTrainFlatten = Tensor<Float>(xTrain.scalars)
print(xTrainFlatten.shape)

[3920]


In [27]:
let xTrain = xTrainFlatten.reshaped(to: [5,784])
print(xTrain.shape)

[5, 784]


## Other arithmetic

In [0]:
let small = Tensor<Float>([[1, 2],
                           [3, 4]])
print(pow(small,2))

[[ 1.0,  4.0],
 [ 9.0, 16.0]]


In [0]:
sqrt(pow(small,2))

[[1.0, 2.0],
 [3.0, 4.0]]


In [0]:
small.sum(alongAxes: 0)

[[4.0, 6.0]]


In [0]:
small.sum(alongAxes: 1)

[[3.0],
 [7.0]]


In [0]:
let temp = Tensor<Float>(ones: [2,3,4])
print(temp.sum(alongAxes: 1).shape)
print(temp.sum(squeezingAxes: 1).shape)

[2, 1, 4]
[2, 4]


In [0]:
var a = Tensor([10.0, 6, -4])
var b = Tensor([2.0, 8, 7]) 

In [0]:
a<b

false


In [0]:
// elementwise operation
print(a .< b)
//merge boolean
print((a.<b).all()) //AND 
print((a.<b).any()) //OR

[false,  true,  true]
false
true


## Broadcasting/ Adding None axis

In [0]:
var a = Tensor<Float>(ones: [3,4])
a+2

[[3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0]]


In [0]:
print(a.expandingShape(at: 0).shape)
print(a.expandingShape(at: 1).shape)
print(a.expandingShape(at: 2).shape)
print(a.expandingShape(at: 3).shape)

[1, 3, 4]
[3, 1, 4]
[3, 4, 1]
Fatal error: Tried to expand dim index 3 for tensor with 2 dimensions.: file /swift-base/swift/stdlib/public/TensorFlow/CompilerRuntime.swift, line 2123
Current stack trace:
0    libswiftCore.so                    0x00007fb130b654a0 _swift_stdlib_reportFatalErrorInFile + 115
1    libswiftCore.so                    0x00007fb130aad30c <unavailable> + 3035916
2    libswiftCore.so                    0x00007fb130aad3fe <unavailable> + 3036158
3    libswiftCore.so                    0x00007fb1308f46c2 <unavailable> + 1230530
4    libswiftCore.so                    0x00007fb130a7a292 <unavailable> + 2826898
5    libswiftCore.so                    0x00007fb1308f3ba9 <unavailable> + 1227689
6    libswiftTensorFlow.so              0x00007fb12d022572 <unavailable> + 599410
7    libswiftTensorFlow.so              0x00007fb12d020cc0 checkOk(_:file:line:) + 508
8    libswiftTensorFlow.so              0x00007fb12d045ad0 _TFCCheckOk(_:) + 81
9    libswiftTensorFlow.so    

: ignored

In [0]:
var a = Tensor<Float>(ones: [5,3,8,8])
var b = Tensor<Float>([1,2,3])
print(a.shape)
print(b.shape)

[5, 3, 8, 8]
[3]


In [0]:
// example of using mean and std to normalize batch of images (8x8x3)
var temp = a + b.expandingShape(at:0).expandingShape(at:2).expandingShape(at:3)
print(temp.shape)

[5, 3, 8, 8]


In [0]:
temp[0,0]

[[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]]


In [0]:
temp[0,1]

[[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0]]


## Using slice to extract column

In [0]:
var a = Tensor<Float>(randomNormal: [3,4])
print(a)


[[-0.51247907,  0.70031214,  0.21330795,  0.21833527],
 [ -1.3798575,   0.7057063, -0.23102938,   2.1162198],
 [  1.6297858,   0.8560183,    0.851844,  0.10909042]]


In [0]:
a.slice(lowerBounds: [0,0],upperBounds: [a.shape[0],0+1])

[[-0.51247907],
 [ -1.3798575],
 [  1.6297858]]


In [0]:
a.slice(lowerBounds: [0,0],upperBounds: [a.shape[0],0+1]).squeezingShape(at: 1)

[-0.51247907,  -1.3798575,   1.6297858]


#  Matmul from scratch

## 1D array input

In [0]:
func matmul(a: [Float], b: [Float], aDim: (Int, Int), bDim: (Int, Int)) -> [Float] {
  
  assert(aDim.1 == bDim.0, "matrix mult shape mismatch")
  var res = Array<Float>(repeating: 0.0, count: aDim.0 * bDim.1)
  for i in 0..<aDim.0 {
    for j in 0..<bDim.1 {
      for k in 0..<aDim.1 {
        res[i*bDim.1 + j] += a[i*aDim.1 + k] * b[j + k*bDim.1] 
      }
    }
  }
  
  return res
}

In [0]:
let a: [Float] = [0,1,2,3,4,5]
let b: [Float] = [6,7,8,9,10,11,12,13]

In [0]:
let c = matmul(a: a, b: b, aDim: (3,2), bDim: (1,4))
print(c)

Assertion failed: matrix mult shape mismatch: file <Cell 36>, line 3
Current stack trace:
0    libswiftCore.so                    0x00007fb130b654a0 _swift_stdlib_reportFatalErrorInFile + 115
1    libswiftCore.so                    0x00007fb130aad30c <unavailable> + 3035916
2    libswiftCore.so                    0x00007fb130aad3fe <unavailable> + 3036158
3    libswiftCore.so                    0x00007fb1308f46c2 <unavailable> + 1230530
4    libswiftCore.so                    0x00007fb130a7a292 <unavailable> + 2826898
5    libswiftCore.so                    0x00007fb1308f3ba9 <unavailable> + 1227689


: ignored

In [0]:
let c = matmul(a: a, b: b, aDim: (3,2), bDim: (2,4))
print(c)

[10.0, 11.0, 12.0, 13.0, 42.0, 47.0, 52.0, 57.0, 74.0, 83.0, 92.0, 101.0]


In [0]:
let xTrain = Tensor<Float>(randomNormal: [5, 784])
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784) // Xavier
print(weights[0])

[ -0.009449077,  -0.007757622, -0.0014349011,   0.025640832,   0.015581535,   0.039894976,
   -0.00579608,  -0.037376884,   0.019090028,  0.0033364035]


In [0]:
xTrain[0][0..<5]

[ 0.31076983,    1.952319,   -0.771446, -0.33013448,    1.465432]


In [0]:
//flatten
xTrain[0..<5].scalars.count == 5*784 

true


In [0]:
let flatA = xTrain[0..<5].scalars
let flatB = weights.scalars
let (aDims,bDims) = ((5, 784), (784, 10))

In [0]:
time(repeating: 100) {
    _ = matmul(a: flatA, b: flatB, aDim: aDims, bDim: bDims)
}

average: 0.11933654999999996 ms,   min: 0.110525 ms,   max: 0.491485 ms


## Bypass memory safe check (C performance)

In [0]:
func matmulUnsafe(a: UnsafePointer<Float>, b: UnsafePointer<Float>, aDims: (Int,Int), bDims: (Int,Int)) -> [Float] {
    assert(aDims.1 == bDims.0, "matmul shape mismatch")
    
    var res = Array(repeating: Float(0.0), count: aDims.0 * bDims.1)
    // THIS IS ALL YOU NEED
    // res is now treated like a raw C pointer instead of a bounds-checked array
    res.withUnsafeMutableBufferPointer { res in 
        for i in 0 ..< aDims.0 {
            for j in 0 ..< bDims.1 {
                for k in 0 ..< aDims.1 {
                    res[i*bDims.1+j] += a[i*aDims.1+k] * b[k*bDims.1+j]
                }
            }
        }
    }
    return res
}

In [0]:
let flatA = xTrain[0..<5].scalars
let flatB = weights.scalars
let (aDims,bDims) = ((5, 784), (784, 10))

In [0]:
time(repeating: 100) {
    _ = matmulUnsafe(a: flatA, b: flatB, aDims: aDims, bDims: bDims)
}

average: 0.053170679999999984 ms,   min: 0.049704 ms,   max: 0.088783 ms


## Tensor input (2D)

In [0]:
func tensorMatmul(_ a: Tensor<Float>, _ b: Tensor<Float>) -> Tensor<Float> {
    var res = Tensor<Float>(zeros: [a.shape[0], b.shape[1]])

    for i in 0 ..< a.shape[0] {
        for j in 0 ..< b.shape[1] {
            for k in 0 ..< a.shape[1] {
                res[i, j] += a[i, k] * b[k, j]
            }
        }
    }
    return res
}



In [0]:
let xTrain = Tensor<Float>(randomNormal: [5, 784])
var weights = Tensor<Float>(randomNormal: [784, 10]) / sqrt(784) // Xavier
_ = tensorMatmul(xTrain,weights )

Note: This will be really slow because Tensor are very good at **bulk** data processing, not one float at a time

# Timing tensorflow built-in matmul

## On GPU

In [0]:
func timeMatmulTensor(size: Int) {
    var matrix = Tensor<Float>(randomNormal: [size, size])
    print("\n\(size)x\(size):\n  ⏰", terminator: "")
    time(repeating: 10) { 
        let matrix = matrix • matrix 
        _ = matrix[0, 0].scalar
    }
}

timeMatmulTensor(size: 1)     // Tiny
timeMatmulTensor(size: 10)    // Bigger
timeMatmulTensor(size: 100)   // Even Bigger
timeMatmulTensor(size: 1000)  // Biggerest
timeMatmulTensor(size: 5000)  // Even Biggerest


1x1:
  ⏰average: 0.23214700000000002 ms,   min: 0.148317 ms,   max: 0.420043 ms

10x10:
  ⏰average: 0.1973846 ms,   min: 0.172543 ms,   max: 0.223508 ms

100x100:
  ⏰average: 0.18139970000000002 ms,   min: 0.170354 ms,   max: 0.199586 ms

1000x1000:
  ⏰average: 1.1136526999999998 ms,   min: 1.074888 ms,   max: 1.249788 ms

5000x5000:
  ⏰average: 65.015166 ms,   min: 59.549035 ms,   max: 89.100711 ms


## On CPU

In [0]:
withDevice(.cpu) {
    timeMatmulTensor(size: 1)     // Tiny
    timeMatmulTensor(size: 10)    // Bigger
    timeMatmulTensor(size: 100)   // Even Bigger
    timeMatmulTensor(size: 1000)  // Biggerest
    timeMatmulTensor(size: 5000)  // Even Biggerest
}


1x1:
  ⏰average: 0.0427889 ms,   min: 0.034711 ms,   max: 0.073164 ms

10x10:
  ⏰average: 0.07212719999999999 ms,   min: 0.056417 ms,   max: 0.113526 ms

100x100:
  ⏰average: 0.1342534 ms,   min: 0.104488 ms,   max: 0.243828 ms

1000x1000:
  ⏰average: 27.389434599999998 ms,   min: 25.955144 ms,   max: 30.90808 ms

5000x5000:
  ⏰average: 3449.5693146999997 ms,   min: 3333.509052 ms,   max: 3932.528158 ms


# Export

Searching for a specific pattern with a regular expression isn't easy in swift. The good thing is that with an extension, we can make it easy for us!

In [0]:
// export
public extension String {
    func findFirst(pat: String) -> Range<String.Index>? {
        return range(of: pat, options: .regularExpression)
    }
    func hasMatch(pat: String) -> Bool {
        return findFirst(pat:pat) != nil
    }
}

The foundation library isn't always the most convenient to use... This is how the first line of the following cell is written in it.

```swift
let url_fname = URL(fileURLWithPath: fname)
let last = fname.lastPathComponent
let out_fname = (url_fname.deletingLastPathComponent().appendingPathComponent("FastaiNotebooks", isDirectory: true)
     .appendingPathComponent("Sources", isDirectory: true)
     .appendingPathComponent("FastaiNotebooks", isDirectory: true).appendingPathComponent(last)
     .deletingPathExtension().appendingPathExtension("swift"))
```

This function parses the underlying json behind a notebook to keep the code in the cells marked with `//export`.

In [0]:
//export
public func notebookToScript(fname: Path){
    let newname = fname.basename(dropExtension: true)+".swift"
    let url = fname.parent/"FastaiNotebooks/Sources/FastaiNotebooks"/newname
    do {
        let data = try Data(contentsOf: fname.url)
        let jsonData = try JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = jsonData["cells"] as! [[String:Any]]
        var module = """
/*
THIS FILE WAS AUTOGENERATED! DO NOT EDIT!
file to edit: \(fname.lastPathComponent)

*/
        
"""
        for cell in cells {
            if let source = cell["source"] as? [String], !source.isEmpty, 
                   source[0].hasMatch(pat: #"^\s*//\s*export\s*$"#) {
                module.append("\n" + source[1...].joined() + "\n")
            }
        }
        try module.write(to: url, encoding: .utf8)
    } catch {
        print("Can't read the content of \(fname)")
    }
}

And this will do all the notebooks in a given folder.

In [0]:
// export
public func exportNotebooks(_ path: Path) {
    for entry in try! path.ls()
    where entry.kind == Entry.Kind.file && 
          entry.path.basename().hasMatch(pat: #"^\d*_.*ipynb$"#) {
        print("Converting \(entry)")
        notebookToScript(fname: entry.path)
    }
}

In [0]:
notebookToScript(fname: Path.cwd/"00_load_data.ipynb")

But now that we implemented it from scratch we're allowed to use it as a package ;). NotebookExport has been written by pcuenq
and will make our lives easier.

In [0]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/"00_load_data.ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

success
